In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pprint import pprint
import json

In [ ]:
path = "data/MERGED2015_16_PP.csv"
data = pd.read_csv(path, low_memory=False)

path_2014 = "data/MERGED2013_14_PP.csv"
data_2014 = pd.read_csv(path_2014, low_memory=False)

# Data Preprocessing

### Listing out columns of importance

In [ ]:
cols = ["OPEID", "UNITID", "INSTNM", "CITY", "STABBR", "ZIP", "CURROPER", "MAIN", "PREDDEG", "HIGHDEG", 
        "CONTROL", "RELAFFIL", "DISTANCEONLY", "ADM_RATE", "SAT_AVG", "ACTCMMID", "UGDS", "UGDS_WHITE", "UGDS_BLACK", "UGDS_HISP", 
        "UGDS_ASIAN", "UGDS_AIAN", "UGDS_NHPI", "UGDS_2MOR", "UGDS_NRA", "UGDS_UNKN", 
        "HBCU", "PBI", "ANNHI", "TRIBAL", "HSI", "NANTI", "MENONLY", "WOMENONLY", "PPTUG_EF", 
        "UG25ABV", "INC_PCT_LO", "INC_PCT_M1", "INC_PCT_M2", "INC_PCT_H1", "INC_PCT_H2", 
        "PAR_ED_PCT_1STGEN", "NPT4_PUB", "NPT4_PRIV", "COSTT4_A", "TUITIONFEE_IN", "TUITIONFEE_OUT", 
        "NPT41_PUB", "NPT42_PUB", "NPT43_PUB", "NPT44_PUB", "NPT45_PUB", "NPT41_PRIV", "NPT42_PRIV", 
        "NPT43_PRIV", "NPT44_PRIV", "NPT45_PRIV", "PCTFLOAN", "PCTPELL", "GRAD_DEBT_MDN", 
        "WDRAW_DEBT_MDN", "GRAD_DEBT_MDN10YR", "CDR3", "RPY_3YR_RT", 
        "RPY_5YR_RT", "RPY_7YR_RT", "C150_4", "D150_4", "CIP01BACHL", "CIP03BACHL", "CIP04BACHL", "CIP05BACHL", "CIP09BACHL", 
        "CIP10BACHL", "CIP11BACHL", "CIP12BACHL", "CIP13BACHL", "CIP14BACHL", "CIP15BACHL", 
        "CIP16BACHL", "CIP19BACHL", "CIP22BACHL", "CIP23BACHL", "CIP24BACHL", "CIP25BACHL", 
        "CIP26BACHL", "CIP27BACHL", "CIP29BACHL", "CIP30BACHL", "CIP31BACHL", "CIP38BACHL", 
        "CIP39BACHL", "CIP40BACHL", "CIP41BACHL", "CIP42BACHL", "CIP43BACHL", "CIP44BACHL", 
        "CIP45BACHL", "CIP46BACHL", "CIP47BACHL", "CIP48BACHL", "CIP49BACHL", "CIP50BACHL", 
        "CIP51BACHL", "CIP52BACHL", "CIP54BACHL", "PCIP01", "PCIP03", "PCIP04", "PCIP05", 
        "PCIP09", "PCIP10", "PCIP11", "PCIP12", "PCIP13", "PCIP14", "PCIP15", "PCIP16", 
        "PCIP19", "PCIP22", "PCIP23", "PCIP24", "PCIP25", "PCIP26", "PCIP27", "PCIP29", 
        "PCIP30", "PCIP31", "PCIP38", "PCIP39", "PCIP40", "PCIP41", "PCIP42", "PCIP43", 
        "PCIP44", "PCIP45", "PCIP46", "PCIP47", "PCIP48", "PCIP49", "PCIP50", "PCIP51", 
        "PCIP52", "PCIP54"]

In [ ]:
df = data[cols]
df = df.loc[(df['PREDDEG'] == 3) & (df['CURROPER'] == 1)]
df.shape

### Merging the Earnings data from 2013-14 dataset

In [ ]:
data_2014 = data_2014.loc[data_2014['PREDDEG'] == 3]
earn_cols = ["UNITID", "MD_EARN_WNE_P6", "MD_EARN_WNE_P10"]
df_earn = data_2014[earn_cols]

In [ ]:
df = pd.merge(df, df_earn, on='UNITID', how='left')
cols = cols+["MD_EARN_WNE_P6", "MD_EARN_WNE_P10"]

### Filtering data for just Undergrad programs, correcting data types of columns and getting rid of unwanted literals

In [ ]:
df.drop(["PREDDEG", "CURROPER"], axis=1, inplace=True)
df = df.replace('PrivacySuppressed', df.replace(['PrivacySuppressed'], [None]))
wrong_data_type_cols = ["INC_PCT_LO", "INC_PCT_M1", "INC_PCT_M2", "INC_PCT_H1", "INC_PCT_H2", 
                        "PAR_ED_PCT_1STGEN", "GRAD_DEBT_MDN", "WDRAW_DEBT_MDN", 
                        "GRAD_DEBT_MDN10YR", "RPY_3YR_RT", "RPY_5YR_RT", "RPY_7YR_RT", "MD_EARN_WNE_P6", "MD_EARN_WNE_P10"]
df[wrong_data_type_cols] = df[wrong_data_type_cols].apply(pd.to_numeric)

## Filling NaN values

In [ ]:
def fill_nan(col):
    if df[col].dtype != 'O':
        if col in ["MD_EARN_WNE_P6", "MD_EARN_WNE_P10"]:
            df[col].fillna(0, inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)

### Filling in the SAT_AVGs from merged Scorecard.csv

In [ ]:
sat_scores = pickle.load(open("final_sat.p", "rb"))
final_sat = {}
for key in sat_scores:
    if not any(c.isalpha() for c in key):
        final_sat[int(key)] = sat_scores[key]

In [ ]:
for opeid in final_sat:
    df.loc[(df['OPEID'] == opeid), 'SAT_AVG'] = final_sat[opeid]

### Merging the financial data columns for Public and Private Institutions and updating column names

In [ ]:
cols_pub = ["NPT4_PUB", "NPT41_PUB", "NPT42_PUB", "NPT43_PUB", "NPT44_PUB", "NPT45_PUB"]
cols_priv = ["NPT4_PRIV", "NPT41_PRIV", "NPT42_PRIV", "NPT43_PRIV", "NPT44_PRIV", "NPT45_PRIV"]
for i in range(len(cols_pub)):
    df[cols_pub[i]] = df[cols_pub[i]].fillna(df[cols_priv[i]])
    
df.rename(columns={"NPT4_PUB" : "NPT4", "NPT41_PUB" : "NPT41", "NPT42_PUB" : "NPT42", 
                   "NPT43_PUB" : "NPT43", "NPT44_PUB" : "NPT44", "NPT45_PUB" : "NPT45"}, 
          inplace=True)
df.drop(cols_priv, axis=1, inplace=True)

### Filling NaN values for all financial data columns with mean values based on CONTROL type

In [ ]:
for i in range(1,4):
    for col in ['COSTT4_A', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'NPT4', 'NPT41', 'NPT42', 'NPT43', 'NPT44', 'NPT45']:
        df.loc[(df['CONTROL'] == i), col] = df.loc[(df['CONTROL'] == i)][col].fillna(int(df.loc[(df['CONTROL'] == i)][col].mean()))

### Filling the remaining NaN values with median values

In [ ]:
#For religious affiliation, if value not present then marking it as "Not Reported (-1)"
df['RELAFFIL'] = df['RELAFFIL'].fillna(-1)

for col in df.columns:
    fill_nan(col)

### Cleaning CIP columns

In [ ]:
cip_columns = ["CIP01BACHL", "CIP03BACHL", "CIP04BACHL", "CIP05BACHL", "CIP09BACHL", 
        "CIP10BACHL", "CIP11BACHL", "CIP12BACHL", "CIP13BACHL", "CIP14BACHL", "CIP15BACHL", 
        "CIP16BACHL", "CIP19BACHL", "CIP22BACHL", "CIP23BACHL", "CIP24BACHL", "CIP25BACHL", 
        "CIP26BACHL", "CIP27BACHL", "CIP29BACHL", "CIP30BACHL", "CIP31BACHL", "CIP38BACHL", 
        "CIP39BACHL", "CIP40BACHL", "CIP41BACHL", "CIP42BACHL", "CIP43BACHL", "CIP44BACHL", 
        "CIP45BACHL", "CIP46BACHL", "CIP47BACHL", "CIP48BACHL", "CIP49BACHL", "CIP50BACHL", 
        "CIP51BACHL", "CIP52BACHL", "CIP54BACHL"]

In [ ]:
for col in cip_columns:
    df[col] = df[col].clip_upper(1)

## Crime Rate Calculation

In [ ]:
def calc_crime_rate(dfc):
    dfc['ZIP'] = dfc['ZIP'].apply(lambda x: str(x)[:5])
    dfc = dfc.fillna(0)
    remove_col = ['UNITID_P','INSTNM','BRANCH','Address','City','State','ZIP','sector_cd','Sector_desc',
                  'men_total','women_total','Total','FILTER14','FILTER15', 'FILTER16','crime_count']
    cols = list(dfc.columns.values)
    cols = [ x for x in cols if x not in remove_col]
    dfc['crime_count'] = dfc[cols].sum(axis=1)
    return dfc

#### On Campus crime

In [ ]:
df_crime = pd.read_excel("data/oncampuscrime141516.xls")
print("Calculating crime rate...")
df_crime = calc_crime_rate(df_crime)
print("Writing Crime Rate into File...")
df_crime.to_excel("data/oncampuscrime141516.xls", encoding='utf-8', index=False)

#### Non Campus crime

In [ ]:
df_crime = pd.read_excel("data/noncampuscrime141516.xls")
print("Calculating crime rate...")
df_crime = calc_crime_rate(df_crime)
print("Writing Crime Rate into File...")
df_crime.to_excel("data/noncampuscrime141516.xls", encoding='utf-8', index=False)
print("Successful")

#### Merge Data set with crime rate

In [ ]:
dfnc = pd.read_excel("data/noncampuscrime141516.xls")
dfc = pd.read_excel("data/oncampuscrime141516.xls")
dfnc = dfnc[['ZIP','crime_count','State']]
dfc = dfc[['ZIP','crime_count','State']]

In [ ]:
dfc = pd.merge(dfc, dfnc, on='ZIP')
dfc.head()
dfc['crime_count'] = dfc['crime_count_x'] + dfc['crime_count_y']

In [ ]:
dfcrime = dfc[['ZIP','crime_count','State_x']]
dfcrime.head()

#### Load Data set

In [ ]:
dfdata = df.copy()
# print(dfdata.columns.values)
dfdata.shape
dfcrime= dfcrime.drop_duplicates('ZIP')

#### Clean Zip code in original Data set and crime rate data 

In [ ]:
dfdata['ZIP'] = dfdata['ZIP'].apply(lambda x: x.zfill(5))
dfdata['ZIP'] = dfdata['ZIP'].apply(lambda x: x[:5])
dfcrime['ZIP'] = dfcrime['ZIP'].apply(lambda x: str(x).zfill(5))
dfcrime['ZIP'] = dfcrime['ZIP'].apply(lambda x: x[:5])
dfd = pd.merge(dfdata, dfcrime, on="ZIP", how="left")

In [ ]:
# Fill Null crime_count with 0
dfd['crime_count'] = dfd['crime_count'].fillna(0)
print("Null: "+str(dfd['crime_count'].isnull().sum()))
dfavg = dfd.loc[dfd['crime_count']!=0]
# Calculate Crime rate average per State
dfdd = dfavg.groupby('State_x', as_index=False)['crime_count'].mean()
dfd = pd.merge(dfd, dfdd, left_on="STABBR", right_on = "State_x", how='left')
print(dfd.head())
# Fill Crime rate average in missing data
dfd['crime_count_y'] = dfd['crime_count_y'].fillna(0)
dfd.loc[dfd['crime_count_x']==0, 'crime_count_x'] = dfd['crime_count_y']
print("Zero: "+str(len(dfd.loc[dfd['crime_count_x']==0])))
print("Null: "+str(dfd['crime_count_x'].isnull().sum()))
dfd.drop(['State_x_x','State_x_y','crime_count_y'], axis=1, inplace=True)

In [ ]:
print("Writing Crime Rate into dataset")
df['CRIME_COUNT'] = dfd['crime_count_x']
df.head()

### Function to fetch Temperature data from temperatures.json and add it to CSV

In [ ]:
def get_temperatures(df_temp):
    d=json.load(open('temperatures.json'))
    temps=[]
    zips=df['ZIP'].tolist()
    zips=[zips[i].split("-")[0] for i in range(len(zips))]
    for i in range(len(zips)):
        temps.append(d[zips[i]])
    cols=np.asarray(temps,dtype=float)
    df_temp['SPRING_TAVG'], df_temp['SUMMER_TAVG'], df_temp['FALL_TAVG'], df_temp['WINTER_TAVG'] = cols[:,0], cols[:,1], cols[:,2], cols[:,3]
    return df_temp

In [ ]:
df = get_temperatures(df)
df.head()

### Saving the cleaned data to a new file

In [ ]:
df.to_csv("cleaned_data.csv", encoding='utf-8', index=False)